# OCT Numerical model

In [2]:
import miepython
import numpy as np
import matplotlib.pyplot as plt
import DataProcessingOCT
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [3]:
# Here some basic inputs
centre_wavelength = 800*10**(-9)
bandwidth_wavelength = 50e-9 # meter 
input_power = 0.001 # watt
integration_time = 1e-3 # seconds
alpha = 0.5 # intensity splitting ratio
num_samples = 1024

upsample = 8

In [4]:
# calculate some variables based on the initial parameters
kc = 2*np.pi/centre_wavelength
dk = bandwidth_wavelength *2*np.pi/centre_wavelength**2

#FWHM_k = bandwidth_wavelength*2*np.pi/centre_wavelength**2
sigma_k = dk/(2*np.sqrt(2*np.log(2)))

# minimum en maximum waardes van k
k_min = kc-1.5*dk
k_max = kc+1.5*dk

# het maken van de k_space
k_space = np.linspace(k_min,k_max,num_samples)

delta_k_space = k_space[1]-k_space[0]

# create the z_space
z_space = np.linspace(-0.5*np.pi/((k_max-k_min)/num_samples),0.5*np.pi/((k_max-k_min)/num_samples),num_samples)

# create the input signal in k space
signal_k_space = input_power*delta_k_space/np.sqrt(2*np.pi*sigma_k**2)*np.exp(-(k_space-kc)**2/(2*sigma_k**2))



In [5]:
# define some more parameters and retrieve the scattering efficiacy
z_surface = 200e-6

n_part = 1.38
n_med = 1.33
volume = 0.01

particle_diameter = 2000*10**(-9)

# number of particles 
Conc = volume/((4/3) * np.pi * (particle_diameter/2)**3)

a,Q,c,d = miepython.ez_mie(n_part,particle_diameter,centre_wavelength,n_med)

print('De scattering efficiacy is ',Q)
mu_s = Conc * Q * np.pi * (particle_diameter/2)**2 

L = 25e-6
D = 0.8e-3
P_NA = 0.5
N_iter = 100

number_of_particles = round(Conc*D*L**2)

particles_per_bin = number_of_particles*(z_space[1]-z_space[0])/D
R_part = P_NA * Q * np.pi*(particle_diameter/2)**2

print(number_of_particles)

De scattering efficiacy is  0.30153937437282
1194


In [6]:
zs = z_space
M = num_samples
zsurf = z_surface

# B-scan

Create a B scan based on creating multiple A-scans and arranging them in an array.


In [7]:
# Define some parameters for the simulation
N_lim = 1
N_A_scans = 512   # number of A scans

# create the image array
speckle_image = np.zeros([int(M/2),N_A_scans])

# iterate over all a scans
for a_scan_num in range(N_A_scans):

    print('Calculating A Scan: ',a_scan_num)

    # create a new particle distribution for each a scan
    z_part = z_surface + np.random.uniform(0,1,number_of_particles)*D
    z_part = np.sort(z_part)

    # create an array with the count of all particles
    cnt = np.arange(number_of_particles)

    # calculate the transmission coefficient for each particle
    transm = (1-(Q*np.pi*(particle_diameter/2)**2/L**2))**(cnt)

    # define and create a phase factor for each particle
    phase_factors = np.zeros([num_samples,number_of_particles],dtype=complex)

    for a in np.arange(1,number_of_particles):
        phase_factors[:,a] = np.sqrt(R_part)*transm[a-1]*np.exp(1j*2*z_part[a]*k_space)

    phase_factors = np.sum(phase_factors,1)

    # calculate the fields due to the reference and the sample
    Ek_sample = np.sqrt(alpha*(1-alpha))*np.sqrt(signal_k_space)*phase_factors
    Ek_ref = np.sqrt(alpha*(1-alpha))*np.sqrt(signal_k_space)

    # using the fields calculate the intensity
    Ik_int = (Ek_sample+Ek_ref)*np.conj(Ek_sample+Ek_ref) - Ek_sample*np.conj(Ek_sample)-Ek_ref*np.conj(Ek_ref)

    # calculate the OCT signal
    OCT_signal = np.fft.ifftshift(np.fft.ifft(Ik_int))

    # place a scan in the matrix
    speckle_image[:,a_scan_num] = np.abs(OCT_signal[int(M/2):])




Calculating A Scan:  0
Calculating A Scan:  1
Calculating A Scan:  2
Calculating A Scan:  3
Calculating A Scan:  4
Calculating A Scan:  5
Calculating A Scan:  6
Calculating A Scan:  7
Calculating A Scan:  8
Calculating A Scan:  9
Calculating A Scan:  10
Calculating A Scan:  11
Calculating A Scan:  12
Calculating A Scan:  13
Calculating A Scan:  14
Calculating A Scan:  15
Calculating A Scan:  16
Calculating A Scan:  17
Calculating A Scan:  18
Calculating A Scan:  19
Calculating A Scan:  20
Calculating A Scan:  21
Calculating A Scan:  22
Calculating A Scan:  23
Calculating A Scan:  24
Calculating A Scan:  25
Calculating A Scan:  26
Calculating A Scan:  27
Calculating A Scan:  28
Calculating A Scan:  29
Calculating A Scan:  30
Calculating A Scan:  31
Calculating A Scan:  32
Calculating A Scan:  33
Calculating A Scan:  34
Calculating A Scan:  35
Calculating A Scan:  36
Calculating A Scan:  37
Calculating A Scan:  38
Calculating A Scan:  39
Calculating A Scan:  40
Calculating A Scan:  41
Ca

In [ ]:
# Plot the image using the plotting tools in DataProcessingOCT
import DataProcessingOCT

fig1,ax=DataProcessingOCT.plot_Bscan_image(speckle_image,dBlevel=25,title='Speckle Image')


# Flow in a channel over multiple B-scans

In [ ]:
# set some parameters for the measurement and the flow
speed_of_flow = 2e-3 # meters per second
a_scan_rate = 5e3 # hertz
N_A_scans = 512   # number of A scans per B scan
N_B_scans = 50 # number of b scans

# from this calculate the b scan time:
b_scan_time = 1/a_scan_rate*N_A_scans

# otherwise set the b scan time manually
b_scan_time = b_scan_time  # in that case change this

print('B scan time is: ',b_scan_time)

# from flow speed to pixels per b scan
pixels_per_second = round(speed_of_flow * 1/L)
pixels_per_b_scan = round(pixels_per_second*b_scan_time)

print("The amount of pixels moved after a b scan is: ",pixels_per_b_scan)

# define the location of the flow channel in the sample
z_min_channel = 0.4e-3
z_max_channel = 0.8e-3

# create an array for all particle positions over all scans
particle_positions = np.zeros([N_A_scans,number_of_particles,N_B_scans])

# number of particles per area:
num_part1 = int(round(number_of_particles/D*(z_min_channel-z_surface)))  # number of particles in area 1 calculated by size
num_part2 = int(round(number_of_particles/D*(z_max_channel-z_min_channel)))+1 # the plus one is due to rounding down errors
num_part3 = int(round(number_of_particles/D*(D+z_surface-z_max_channel)))

# for area 1,2 en 3 allocate the positions randomly
# create arrays for all positions in all a scans
z_particles_1 = np.zeros([N_A_scans,num_part1])
z_particles_2 = np.zeros([N_A_scans,num_part2,N_B_scans])
z_particles_3 = np.zeros([N_A_scans,num_part3])

# create the first particle positions
for A_scan in range(N_A_scans):

    z_particles_1[A_scan,:] = z_surface + np.random.uniform(0,1,num_part1)*(z_min_channel-z_surface)
    
    z_particles_3[A_scan,:] = z_max_channel + np.random.uniform(0,1,num_part3)*(D+z_surface-z_max_channel)

    # create the particles in the channel at t=0
    z_particles_2[A_scan,:,0] = z_min_channel + np.random.uniform(0,1,num_part2)*(z_max_channel-z_min_channel)
    
    # allocate the particle positions at t=0
    particle_positions[A_scan,:,0] = np.sort(np.concatenate((z_particles_1[A_scan,:],z_particles_2[A_scan,:,0],z_particles_3[A_scan,:])))


# create the effect of flow in over all b scans
for B_scan in range(1,N_B_scans):
    # for every b scan shift the speckle in the channel
    for A_scan in range(pixels_per_b_scan*B_scan):
        part_2 = z_min_channel + np.random.uniform(0,1,num_part2)*(z_max_channel-z_min_channel)
        particle_positions[A_scan,:,B_scan] = np.sort(np.concatenate((z_particles_1[A_scan,:],part_2,z_particles_3[A_scan,:])))


    for A_scan in range(pixels_per_b_scan*B_scan,N_A_scans):

        # create the particle positions for each a scan
        part_2 = z_particles_2[A_scan-pixels_per_b_scan*B_scan,:,0]

        # set the particle positions
        particle_positions[A_scan,:,B_scan] = np.sort(np.concatenate((z_particles_1[A_scan,:],part_2,z_particles_3[A_scan,:])))

        



In [ ]:
plt.imshow(particle_positions[:,:,1])

In [ ]:
# define the function that creates a speckle image based on the particle positions

def create_speckle_image(particle_positions, N_A_scans):

    # create the image array:
    speckle_image = np.zeros([int(M/2),N_A_scans])

    # calculate the a scans
    for a_scan_num in range(N_A_scans):

        # retrieve the particle positions
        z_part = particle_positions[a_scan_num,:]

        # create an array with the count of all particles
        cnt = np.arange(number_of_particles)

        # calculate the transmission coefficient for each particle
        transm = (1-(Q*np.pi*(particle_diameter/2)**2/L**2))**(cnt)

        # define and create a phase factor for each particle
        phase_factors = np.zeros([num_samples,number_of_particles],dtype=complex)

        for a in np.arange(1,number_of_particles):
            phase_factors[:,a] = np.sqrt(R_part)*transm[a-1]*np.exp(1j*2*z_part[a]*k_space)

        phase_factors = np.sum(phase_factors,1)

        # calculate the fields due to the reference and the sample
        Ek_sample = np.sqrt(alpha*(1-alpha))*np.sqrt(signal_k_space)*phase_factors
        Ek_ref = np.sqrt(alpha*(1-alpha))*np.sqrt(signal_k_space)

        # using the fields calculate the intensity
        Ik_int = (Ek_sample+Ek_ref)*np.conj(Ek_sample+Ek_ref) - Ek_sample*np.conj(Ek_sample)-Ek_ref*np.conj(Ek_ref)

        # calculate the OCT signal
        OCT_signal = np.fft.ifftshift(np.fft.ifft(Ik_int))

        # place a scan in the matrix
        speckle_image[:,a_scan_num] = np.abs(OCT_signal[int(M/2):])

    return speckle_image

In [ ]:
# create the timeseries data
image_timeseries = np.zeros([int(M/2),N_A_scans, N_B_scans])
for B_scan in range(N_B_scans):
    print(B_scan)
    image_timeseries[:,:,B_scan] = create_speckle_image(particle_positions[:,:,B_scan],N_A_scans)


In [ ]:
# save the image data
#image_timeseries.shape
#np.save("timeseries512-512-50.npy",image_timeseries)

In [ ]:
# plot the average over the timeseries of the data
image_average = np.average(image_timeseries,2)
fig1,ax=DataProcessingOCT.plot_Bscan_image(image_average,dBlevel=25,title='Average of timeseries flow data')
plt.show()